In [1]:
from transformers import pipeline
import numpy as np
import pandas as pd
from datasets import load_dataset, Audio

In [2]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [3]:
sampling_rate = feature_extractor.sampling_rate
dataset = load_dataset("audiofolder", data_dir="Musicnet+")
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

Resolving data files:   0%|          | 0/476 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset['train'][0]

{'audio': {'path': 'C:\\Users\\young\\OneDrive - Nanyang Technological University\\NTU\\AI Master Project\\Musicnet_extend\\Musicnet+\\Baroque\\Baroque.001.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          4.36930986e-06,  8.86010457e-06, -7.46243472e-07]),
  'sampling_rate': 16000},
 'label': 0}

In [5]:
id2label_fn = dataset["train"].features["label"].int2str
id2label_fn(dataset["train"][2]["label"])

'Baroque'

In [6]:
import math
max_duration = 3 

def subdivide_function(entry):
    audio_array = entry["audio"][0]['array']
    audio_label = entry['label'][0]
    #print(audio_array)
    #print(audio_label)
    sampling_rate = feature_extractor.sampling_rate
    segment_length = int(sampling_rate * max_duration)
    
    #print(len(audio_array))
    num_segments = math.ceil(len(audio_array) / segment_length)
    chunks = []
    labels = []
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        if end_idx >= len(audio_array):
            segment = audio_array[start_idx:] #meaning last segment already
        else:
            segment = audio_array[start_idx:end_idx] #meaning still in the middle
        #print(segment)
        labels.append(audio_label)
        chunks.append(segment)

    return {"labels": labels,
            "chunks": chunks}

chunked_dataset = dataset.map(subdivide_function, batch_size = 1, batched=True, remove_columns=['audio', 'label'])

Map:   0%|          | 0/476 [00:00<?, ? examples/s]

In [7]:
chunked_dataset['train'][0]

{'labels': 0,
 'chunks': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0

In [8]:
chunked_dataset = chunked_dataset["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)

In [9]:
chunked_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'chunks'],
        num_rows: 50434
    })
    test: Dataset({
        features: ['labels', 'chunks'],
        num_rows: 5604
    })
})

In [10]:
def preprocess_function(examples):
    audio_arrays = examples['chunks']
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

dataset_encoded = chunked_dataset.map(
    preprocess_function,
    remove_columns=["chunks"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
dataset_encoded = dataset_encoded.rename_column("labels", "label")
dataset_encoded

Map:   0%|          | 0/50434 [00:00<?, ? examples/s]

Map:   0%|          | 0/5604 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 50434
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 5604
    })
})

In [11]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(dataset["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["0"]
label2id['Baroque']

'0'

In [12]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 4
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-dataset",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=False,
)

C:\Users\young\anaconda3\envs\ClassicalMusicClassify\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [15]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset_encoded["train"].with_format("torch"),
    eval_dataset=dataset_encoded["test"].with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("PeriodClassifyModel")

C:\Users\young\AppData\Local\Temp\ipykernel_40412\2645680901.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.818900,0.637589,0.714133
2,1.048300,0.831706,0.780335
3,0.330200,0.694114,0.837438
4,0.169700,0.734180,0.850286
5,1.150400,0.692392,0.858494
6,0.007200,0.914628,0.859208
7,0.380600,0.902987,0.871877
8,0.000000,0.950044,0.882227
9,0.000000,0.997732,0.890257
10,0.000000,1.106803,0.890079
